# Regressão Linear com PyTorch

## Objetivos

Este exemplo utiliza regressão linear para estimar o comprimento das sépalas da íris a partir do seu comprimento das pétalas.
Utiliza-se:
- a função de perda MSE do PyTorch,
- treinamento dos parâmetros via gradiente descendente usando o otimizador.
- A rede é criada com uma camada nn.Linear()

## Importação dos pacotes

In [1]:
%matplotlib inline
import torch
from torch import nn, optim
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris

torch.manual_seed(1234)

## Leitura dos dados

In [2]:
iris = load_iris()
data = iris.data[iris.target==1,::2]  # comprimento das sépalas e pétalas, indices 0 e 2

x_train = data[:,0:1].astype(np.float32)
y_train = data[:,1:2].astype(np.float32)

n_samples = x_train.shape[0]
print('x_train.shape:',x_train.shape, x_train.dtype)
print('y_train.shape:',y_train.shape, y_train.dtype)

print('x_train[:5]:\n', x_train[:5])
print('y_train[:5]:\n', y_train[:5])

x_train.shape: (50, 1) float32
y_train.shape: (50, 1) float32
x_train[:5]:
 [[7. ]
 [6.4]
 [6.9]
 [5.5]
 [6.5]]
y_train[:5]:
 [[4.7]
 [4.5]
 [4.9]
 [4. ]
 [4.6]]


### Normalização dos dados

In [3]:
x_train -= x_train.min()
x_train /= x_train.max()
y_train -= y_train.min()
y_train /= y_train.max()


In [4]:
x_train_bias = np.hstack([np.ones(shape=(n_samples,1)), x_train])

In [5]:
x_train_bias = torch.FloatTensor(x_train_bias)
y_train      = torch.FloatTensor(y_train)

## Criação do modelo da rede

In [6]:
model = torch.nn.Linear(2, 1, bias=False)

### Verificando a inicialização dos parâmetros

In [14]:
model.weight.data = torch.zeros(1,2)
torch.nn.init.uniform(model.weight.data, -0.1, 0.1)
model.weight.data

<ipython-input-14-e458ff4bc28e>:2: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  torch.nn.init.uniform(model.weight.data, -0.1, 0.1)


tensor([[-0.0896, -0.0064]])

### Testando o predict da rede

In [8]:
model(Variable(torch.ones((5,2))))

tensor([[-0.0747],
        [-0.0747],
        [-0.0747],
        [-0.0747],
        [-0.0747]], grad_fn=<MmBackward0>)

## Treinamento

### Definindo função de perda e otimizador

In [10]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.5)

### Laço de treinamento

In [15]:
num_epochs = 120
w0_list = []
w1_list = []
for epoch in range(num_epochs):
    inputs = Variable(x_train_bias)
    target = Variable(y_train)

    # forward - predict
    out = model(inputs)

    #loss cálculo da função de
    #loss = torch.mean((y_pred - y_train)**2)
    loss = criterion(out, target)
    print(model.weight.data)

    # backward e otimizador
    #model.weight.data = model.weight.data - learning_rate * model.weight.grad.data
    #model.weight.grad.data.zero_()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(model.weight.data)

    # verbose
    # if (epoch+1) % 20 == 0:
    print('Epoch[{}/{}], loss: {:.6f}'
          .format(epoch+1, num_epochs, loss.data))

tensor([[-0.0896, -0.0064]])
tensor([[0.6031, 0.3764]])
Epoch[1/120], loss: 0.529541
tensor([[0.6031, 0.3764]])
tensor([[0.4143, 0.3016]])
Epoch[2/120], loss: 0.062524
tensor([[0.4143, 0.3016]])
tensor([[0.4512, 0.3426]])
Epoch[3/120], loss: 0.031301
tensor([[0.4512, 0.3426]])
tensor([[0.4310, 0.3530]])
Epoch[4/120], loss: 0.028579
tensor([[0.4310, 0.3530]])
tensor([[0.4259, 0.3702]])
Epoch[5/120], loss: 0.027780
tensor([[0.4259, 0.3702]])
tensor([[0.4174, 0.3847]])
Epoch[6/120], loss: 0.027164
tensor([[0.4174, 0.3847]])
tensor([[0.4102, 0.3991]])
Epoch[7/120], loss: 0.026611
tensor([[0.4102, 0.3991]])
tensor([[0.4031, 0.4126]])
Epoch[8/120], loss: 0.026110
tensor([[0.4031, 0.4126]])
tensor([[0.3965, 0.4255]])
Epoch[9/120], loss: 0.025655
tensor([[0.3965, 0.4255]])
tensor([[0.3901, 0.4378]])
Epoch[10/120], loss: 0.025242
tensor([[0.3901, 0.4378]])
tensor([[0.3840, 0.4496]])
Epoch[11/120], loss: 0.024867
tensor([[0.3840, 0.4496]])
tensor([[0.3782, 0.4607]])
Epoch[12/120], loss: 0.024526

## Avaliação

In [ ]:
y_pred = model(Variable(x_train_bias))
plt.plot(x_train, y_train.numpy(), 'ro', label='Original data')
plt.plot(x_train, y_pred.data.numpy(), 'kx-', label='Fitting Line')
plt.show()

# Exercícios

- 1) Calcule o valor da função de custo (MSE) depois da rede treinada, utilizando a
   função `criterion` utilizada no laço de treinamento.

In [ ]:
#todo

<details>
<summary>Mostrar Resultado</summary>

```python
y_pred = model(Variable(x_train_bias))
loss = criterion(y_pred, Variable(y_train))
print(loss)
```

</details>

- 2) Faça igual o exercício do notebook anterior, de plotar um gráfico scatterplot para
   mostrar a evolução dos parâmetros durante o treinamento pelo gradiente descendente.

In [ ]:
# todo

<details>
<summary>Mostrar Resultado</summary>

```python
plt.scatter(w0_list,w1_list)
w0_old = None
for (w0,w1) in zip(w0_list,w1_list):
    if w0_old:
        plt.arrow(w0_old, w1_old, w0-w0_old, w1-w1_old,
                  head_length=0.01,head_width=0.01,shape='full',
                  length_includes_head=True)
    w0_old,w1_old = w0,w1
```

</details>

- 3) Procure sobrepor a função de perda neste gráfico. (exercício mais difícil).

# Aprendizados